In [1]:
"""
Created on Wed Oct 14 2020
@author: Nicole Albern

Get vertical profiles of cloud heating rates for each
cloud class.

Calculate weighting as in
https://github.com/aikovoigt/nawdex-hackathon/blob/master/means/calc_heatingrates_means.ipynb
(version: Nov 6, 2020)
"""

import matplotlib.pyplot as plt
#import matplotlib as mpl
import numpy as np
import xarray as xr

import shutil
import zarr

import sys
sys.path.append('/pf/b/b380490/jupyter_notebooks/nawdex_hackathon')
import dict_nawdexsims
simdict = dict_nawdexsims.simdictionary()
#colordict = dict_nawdexsims.colordictionary()

from nawdexutils import drop_first_day, select_analysis_days

import dask
#dask.config.set(**{'array.slicing.split_large_chunks': True})

from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:35538 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 48 Memory: 67.11 GB


In [2]:
ipath_cloudheat = '/work/bb1018/nawdex-hackathon_pp/ddttemp_rad-from-fluxes/'
expid = 'nawdexnwp-2km-mis-0001'
ds_chr = xr.open_zarr(ipath_cloudheat + expid + \
                              '_ddttemp_rad-from-fluxes_DOM01_ML.zarr')

In [3]:
ds_chr

,Array,Chunk
Bytes,237.15 GB,600.00 MB
Shape,"(97, 75, 8149456)","(2, 75, 1000000)"
Count,442 Tasks,441 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,237.15 GB,600.00 MB
Shape,"(97, 75, 8149456)","(2, 75, 1000000)"
Count,442 Tasks,441 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [4]:
ds_chr = select_analysis_days(ds_chr, expid)

In [5]:
ds_chr

,Array,Chunk
Bytes,176.03 GB,600.00 MB
Shape,"(72, 75, 8149456)","(2, 75, 1000000)"
Count,766 Tasks,324 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,176.03 GB,600.00 MB
Shape,"(72, 75, 8149456)","(2, 75, 1000000)"
Count,766 Tasks,324 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [20]:
ds_chr['chr_lw'] = (ds_chr.isel(time=0).ddt_temp_radlw_fromflux - ds_chr.isel(time=0).ddt_temp_radlwclr_fromflux)

In [21]:
ds_chr['chr_lw'] = (ds_chr.ddt_temp_radlw_fromflux - ds_chr.ddt_temp_radlwclr_fromflux)

In [22]:
ds_chr

,Array,Chunk
Bytes,176.03 GB,600.00 MB
Shape,"(72, 75, 8149456)","(2, 75, 1000000)"
Count,766 Tasks,324 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,176.03 GB,600.00 MB
Shape,"(72, 75, 8149456)","(2, 75, 1000000)"
Count,766 Tasks,324 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [23]:
ds_chr['chr_lw'] = (('time', 'height', 'ncells'), ds_chr.ddt_temp_radlw_fromflux - ds_chr.ddt_temp_radlwclr_fromflux)

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KilledWorker: ("('zarr-getitem-3f52e8733ca7139fcf0ef929a440679a', 14, 0, 8)", <Worker 'tcp://127.0.0.1:40259', name: 4, memory: 0, processing: 127>)

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker


In [16]:
ds_chr

,Array,Chunk
Bytes,176.03 GB,600.00 MB
Shape,"(72, 75, 8149456)","(2, 75, 1000000)"
Count,766 Tasks,324 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,176.03 GB,600.00 MB
Shape,"(72, 75, 8149456)","(2, 75, 1000000)"
Count,766 Tasks,324 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [4]:
'''####################################'''
'''      !!!!! VERSION 3 !!!!!         '''
'''compute area mean for each time step'''
'''then do time mean                   '''
'''store all cloud classes in one file '''
'''####################################'''

##############################################################################
opath = '/scratch/b/b380490/hackathon/'

ipath_grid = '/work/bb1018/icon_4_hackathon/'
ipath_oceanmask = '/work/bb1018/nawdex-hackathon_pp/'
ipath_cloudmask = '/pf/b/b380796/scratch/hackathon/george/'
ipath_cloudheat = '/work/bb1018/nawdex-hackathon_pp/ddttemp_rad-from-fluxes/'

#resolutions = ['80km', '40km', '20km', '10km', '5km', '2km', '2km-shcon']
resolutions = ['2km', '2km-shcon']
#gridres = ['R80000m', 'R40000m', 'R20000m', 'R10000m', 'R5000m', 'R2500m', 'R2500m']
gridres = ['R2500m', 'R2500m']

# cloud classes
ncclass = 8 # number of cloud classes
cclasses = ['High (H)', 'Middle (M)', 'Low (L)',
            'H-M', 'M-L', 'H-L', 'H-M-L', 'clear sky']
cclass_save = ['H', 'M', 'L', 'H-M', 'M-L', 'H-L', 'H-M-L', 'clearsky']

for r, resolution in enumerate(resolutions):
    print(resolution)
    if resolution in ['80km', '40km', '20km']:
        sims = ['0001', '0002', '0003', '0004', '0005',
                '0006', '0007', '0008', '0009', '0010']
    elif resolution == '2km-shcon':
        sims = ['0001-shcon', '0002-shcon', '0005-shcon',
                '0006-shcon', '0007-shcon', '0008-shcon',
                '0009-shcon', '0010-shcon', '0011-shcon',
                '0012-shcon']
        resolution = '2km'
    else: # 10km, 5km, 2km
        sims = ['0001', '0002', '0003', '0004', '0005',
                '0006', '0007', '0008', '0009', '0010', '0011', '0012']
        
    for sim in sims:
        print('  ', sim)
        expid = 'nawdexnwp-' + resolution + '-mis-' + sim

        ##########################################################################
        # read ocean mask
        da_ocean = xr.open_dataset(ipath_oceanmask + '/openoceanmask/' + expid + \
                                   '_openoceanmask.nc')['mask_openocean']
        index = np.where(da_ocean == 1)[0]
        del da_ocean

        ##########################################################################
        # read cell area
        da_cell_area = xr.open_dataset(ipath_grid + '/grids/icon-grid_nawdex_78w40e23n80n_' + \
                                       gridres[r] + '.nc')['cell_area'].rename({'cell': 'ncells'})

        # weight for area mean
        #weights = da_cell_area / (da_cell_area).sum(dim=['ncells'])

        # apply open ocean mask
        weights = da_cell_area.isel(ncells=index)#weights.isel(ncells=index)
        weights.name = "weights"

        del da_cell_area

        ##########################################################################
        # read heating rates
        print('      read heating rates')

        ds_chr = xr.open_zarr(ipath_cloudheat + expid + \
                              '_ddttemp_rad-from-fluxes_DOM01_ML.zarr')
        
 
        # remove first day from dataset
        #ds_chr = drop_first_day(ds_chr)
        
        # get days that are available for all resolutions
        ds_chr = select_analysis_days(ds_chr, expid)
        
        print(ds_chr)
        
        # apply open ocean mask
        ds_chr = ds_chr.isel(ncells=index)
        
        # get longwave and shortwave cloud-radiative heating rates by subtracting
        # all-sky and clear-sky values
        ds_chr['chr_lw'] = (('time', 'height', 'ncells'),
                            ds_chr.ddt_temp_radlw_fromflux - ds_chr.ddt_temp_radlwclr_fromflux)
        ds_chr['chr_sw'] = (('time', 'height', 'ncells'),
                            ds_chr.ddt_temp_radsw_fromflux - ds_chr.ddt_temp_radswclr_fromflux)

        ds_chr = ds_chr.drop_vars(('ddt_temp_radlw_fromflux',
                                   'ddt_temp_radlwclr_fromflux',
                                   'ddt_temp_radsw_fromflux',
                                   'ddt_temp_radswclr_fromflux'))
        
        ##########################################################################
        # read cloud classes
        print('      read cloud class')
        # read cloud class: 
        # cloud class stored every 30 minutes; cloud heating rates stored every hour
        # -> resample cloud class to get it every hour
        da_cclass = xr.open_dataset(ipath_cloudmask + 'nawdexnwp_' + resolution + \
                                    '_cloudclass_mis_' + sim + '.nc').rename({'clch': 'cclass'}).resample(time="1H").nearest(tolerance="5M")['cclass']
  
        # get days that are available for all resolutions
        da_cclass = select_analysis_days(da_cclass, expid)
        
        ##########################################################################
        # calculate time mean and area mean cloud heating rates for cloud classes
        print('      calculate area mean and time mean for cloud classes')

        # compute area mean for each timestep individually, then do time mean
        ntime = len(ds_chr.time.values)
        #print('         ntime ', ntime)
        ds_timesteps = []
        for t in range(ntime):
            #print('da_cclass time ', da_cclass.isel(time=t).time.values)
            #print('ds_chr time    ', ds_chr.isel(time=t).time.values)
            
            # check that time steps for heating rates and cloud classes
            # are the same
            if (da_cclass.isel(time=t).time.values != ds_chr.isel(time=t).time.values):
                print('ERROR IN TIME SELECTION!')
                print('cloud class time:  ' + str(da_cclass.isel(time=t).time.values))
                print('heating rate time: ' + str(ds_chr.isel(time=t).time.values))

            # get cloud classes for specific time step
            cc_time = da_cclass.isel(time=t)
            #print('cc_time shape ', cc_time.shape)
            
            # calculate area mean for specific time step and cloud class
            for c, clas in enumerate(cclasses):
                # get heating rate for cloud class
                ds_chr_cc = ds_chr.isel(time=t).where(cc_time == c+1)
                
                # calculate weighted area mean
                ds_areamean = ds_chr_cc.weighted(weights).mean('ncells').compute()
                #ds_areamean = (ds_chr_cc * weights).sum(dim='ncells').compute()
                
                # store area mean in dataset            
                if c == 0:
                    ds_areameanall = ds_areamean.rename({'chr_lw': 'chr_lw_cclass' + str(c+1),
                                                         'chr_sw': 'chr_sw_cclass' + str(c+1)})
                else:
                    ds_areameanall = ds_areameanall.update(ds_areamean.rename({'chr_lw': 'chr_lw_cclass' + str(c+1),
                                                                               'chr_sw': 'chr_sw_cclass' + str(c+1)}))
                del ds_chr_cc, ds_areamean
            
            # add timestep to timestep list
            ds_timesteps.append(ds_areameanall)

            del c, clas, cc_time, ds_areameanall
        
        del t, ntime
        
        # get time mean
        ds_meanall = xr.concat(ds_timesteps, dim='time').mean('time')
        del ds_timesteps
        
        # save means at nc file
        print('      save data')
        ofile = expid + \
                '_cloudheatingrate_cloudclasses_timemean_areamean_oceanmask_applied.nc'
        ds_meanall.to_netcdf(path=opath + ofile)#, format='NETCDF3_64BIT')#, unlimited_dims='time')
        
        del ds_meanall, ofile
    
        ##########################################################################
        del index, weights
        del expid
        del da_cclass, ds_chr
    del sim

del resolution
print('Done!')

2km
   0001
      read heating rates
<xarray.Dataset>
Dimensions:                     (height: 75, ncells: 8149456, time: 97)
Coordinates:
  * height                      (height) float64 1.0 2.0 3.0 ... 73.0 74.0 75.0
  * time                        (time) datetime64[ns] 2016-09-22 ... 2016-09-26
Dimensions without coordinates: ncells
Data variables:
    ddt_temp_radlw_fromflux     (time, height, ncells) float32 dask.array<chunksize=(2, 75, 1000000), meta=np.ndarray>
    ddt_temp_radlwclr_fromflux  (time, height, ncells) float32 dask.array<chunksize=(2, 75, 1000000), meta=np.ndarray>
    ddt_temp_radsw_fromflux     (time, height, ncells) float32 dask.array<chunksize=(2, 75, 1000000), meta=np.ndarray>
    ddt_temp_radswclr_fromflux  (time, height, ncells) float32 dask.array<chunksize=(2, 75, 1000000), meta=np.ndarray>
<xarray.Dataset>
Dimensions:                     (height: 75, ncells: 8149456, time: 72)
Coordinates:
  * height                      (height) float64 1.0 2.0 3.0 ... 73.

distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting


KilledWorker: ("('zarr-getitem-f1a0d60cca8af6654452a1d39cfb1145', 5, 0, 1)", <Worker 'tcp://127.0.0.1:44147', name: 7, memory: 0, processing: 179>)

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:55322 remote=tcp://127.0.0.1:35260>
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


In [ ]:
'''###############################################'''
''' test if ds.weighted() takes nan's into account'''
'''###############################################'''

##############################################################################
opath = '/scratch/b/b380490/hackathon/'

ipath_grid = '/work/bb1018/icon_4_hackathon/'
ipath_oceanmask = '/work/bb1018/nawdex-hackathon_pp/'
ipath_cloudmask = '/pf/b/b380796/scratch/hackathon/george/'
ipath_cloudheat = '/work/bb1018/nawdex-hackathon_pp/ddttemp_rad-from-fluxes/'

# cloud classes
ncclass = 8 # number of cloud classes
cclasses = ['High (H)', 'Middle (M)', 'Low (L)',
            'H-M', 'M-L', 'H-L', 'H-M-L', 'clear sky']
cclass_save = ['H', 'M', 'L', 'H-M', 'M-L', 'H-L', 'H-M-L', 'clearsky']

# simulation details
resolution = '80km'
r = 0
gridres = ['R80000m']
sim = '0001'
expid = 'nawdexnwp-' + resolution + '-mis-' + sim

##########################################################################
# read ocean mask
da_ocean = xr.open_dataset(ipath_oceanmask + '/openoceanmask/' + expid + \
                           '_openoceanmask.nc')['mask_openocean']
index = np.where(da_ocean == 1)[0]
del da_ocean

##########################################################################
# read cell area
da_cell_area = xr.open_dataset(ipath_grid + '/grids/icon-grid_nawdex_78w40e23n80n_' + \
                               gridres[r] + '.nc')['cell_area'].rename({'cell': 'ncells'})

# apply open ocean mask
weights = da_cell_area.isel(ncells=index)
weights.name = "weights"

del da_cell_area

##########################################################################
# read heating rates
print('read heating rates')

ds_chr = xr.open_zarr(ipath_cloudheat + expid + \
                      '_ddttemp_rad-from-fluxes_DOM01_ML.zarr')

# get days that are available for all resolutions
ds_chr = select_analysis_days(ds_chr, expid)

# apply open ocean mask
ds_chr = ds_chr.isel(ncells=index)

# get longwave and shortwave cloud-radiative heating rates by subtracting
# all-sky and clear-sky values
ds_chr['chr_lw'] = (('time', 'height', 'ncells'),
                    ds_chr.ddt_temp_radlw_fromflux - ds_chr.ddt_temp_radlwclr_fromflux)
ds_chr['chr_sw'] = (('time', 'height', 'ncells'),
                    ds_chr.ddt_temp_radsw_fromflux - ds_chr.ddt_temp_radswclr_fromflux)

ds_chr = ds_chr.drop_vars(('ddt_temp_radlw_fromflux',
                           'ddt_temp_radlwclr_fromflux',
                           'ddt_temp_radsw_fromflux',
                           'ddt_temp_radswclr_fromflux'))
        
##########################################################################
# read cloud classes
print('      read cloud class')
# read cloud class: 
# cloud class stored every 30 minutes; cloud heating rates stored every hour
# -> resample cloud class to get it every hour
da_cclass = xr.open_dataset(ipath_cloudmask + 'nawdexnwp_' + resolution + \
                            '_cloudclass_mis_' + sim + '.nc').rename({'clch': 'cclass'}).resample(time="1H").nearest(tolerance="5M")['cclass']

# get days that are available for all resolutions
da_cclass = select_analysis_days(da_cclass, expid)
        
##########################################################################
# calculate area mean cloud heating rates for cloud class 1
        
# weighted area mean
x = (ds_chr.where(da_cclass == 1)) * 0 + 1
print(x)

# mean based on all grid cells
mean1 = x.weighted(weights).mean('ncells')

#print(da_cclass.shape)

# select weights only for grid points that belong to cloud class
# only works for 
print(weights)
print()
print()

weights_cc = weights.where(da_cclass == 1).fillna(0)#dropna(dim='')
print(weights_cc)
#x_cc = x.isel.dropna()
print()
print()
print(weights.shape, weights_cc.shape)

mean2 = x.weighted(weights_cc).mean('ncells')



print(mean1)
print()
print()
print(mean2)


print(mean1.chr_lw.values.min(), mean1.chr_lw.values.max())
print(mean2.chr_lw.values.min(), mean2.chr_lw.values.max())
print()
print()
print((mean1.chr_lw - mean2.chr_lw).values.min(), (mean1.chr_lw - mean2.chr_lw).values.max())

del mean1, mean2, x, weights_cc


##########################################################################
del index, weights
del da_cclass, ds_chr
del sim, expid, resolution, r, gridres

print('Done!')

In [ ]:
'''#####################################'''
'''      !!!!! VERSION 2.5 !!!!!        '''
'''check if order of computing time mean'''
'''and area mean matters                '''
'''store all cloud classes in one file  '''
'''#####################################'''

##############################################################################
opath = '/scratch/b/b380490/hackathon/'

ipath_grid = '/work/bb1018/icon_4_hackathon/'
ipath_oceanmask = '/work/bb1018/nawdex-hackathon_pp/'
ipath_cloudmask = '/pf/b/b380796/scratch/hackathon/george/'
ipath_cloudheat = '/work/bb1018/nawdex-hackathon_pp/ddttemp_rad-from-fluxes/'

resolutions = ['80km']#, '40km', '20km', '10km', '5km', '2km', '2km-shcon']
gridres = ['R80000m', 'R40000m', 'R20000m', 'R10000m', 'R5000m', 'R2500m', 'R2500m']

# cloud classes
ncclass = 8 # number of cloud classes
cclasses = ['High (H)', 'Middle (M)', 'Low (L)',
            'H-M', 'M-L', 'H-L', 'H-M-L', 'clear sky']
cclass_save = ['H', 'M', 'L', 'H-M', 'M-L', 'H-L', 'H-M-L', 'clearsky']

for r, resolution in enumerate(resolutions):
    print(resolution)
    if resolution in ['80km', '40km', '20km']:
        sims = ['0001', '0002', '0003', '0004', '0005',
                '0006', '0007', '0008', '0009', '0010']
    elif resolution == '2km-shcon':
        sims = ['0001-shcon', '0002-shcon', '0005-shcon',
                '0006-shcon', '0007-shcon', '0008-shcon',
                '0009-shcon', '0010-shcon', '0011-shcon',
                '0012-shcon']
        resolution = '2km'
    else: # 10km, 5km, 2km
        sims = ['0001', '0002', '0003', '0004', '0005',
                '0006', '0007', '0008', '0009', '0010', '0011', '0012']
        
    for sim in sims:
        print('  ', sim)
        expid = 'nawdexnwp-' + resolution + '-mis-' + sim

        ##########################################################################
        # read ocean mask
        da_ocean = xr.open_dataset(ipath_oceanmask + '/openoceanmask/' + expid + \
                                   '_openoceanmask.nc')['mask_openocean']
        index = np.where(da_ocean == 1)[0]
        del da_ocean
        
        ##########################################################################
        # read cell area
        da_cell_area = xr.open_dataset(ipath_grid + '/grids/icon-grid_nawdex_78w40e23n80n_' + \
                                       gridres[r] + '.nc')['cell_area'].rename({'cell': 'ncells'})

        # weight for area mean
        #weights = da_cell_area / (da_cell_area).sum(dim=['ncells'])

        # apply open ocean mask
        weights = da_cell_area.isel(ncells=index)#weights.isel(ncells=index)
        weights.name = "weights"

        del da_cell_area

        ##########################################################################
        # read heating rates
        print('      read heating rates')

        ds_chr = xr.open_zarr(ipath_cloudheat + expid + \
                              '_ddttemp_rad-from-fluxes_DOM01_ML.zarr')
 
        # remove first day from dataset
        #ds_chr = drop_first_day(ds_chr)
        
        # get days that are available for all resolutions
        ds_chr = select_analysis_days(ds_chr, expid)
        
        # apply open ocean mask
        ds_chr = ds_chr.isel(ncells=index)
        
        # get longwave and shortwave cloud-radiative heating rates by subtracting
        # all-sky and clear-sky values
        ds_chr['chr_lw'] = (('time', 'height', 'ncells'),
                            ds_chr.ddt_temp_radlw_fromflux - ds_chr.ddt_temp_radlwclr_fromflux)
        ds_chr['chr_sw'] = (('time', 'height', 'ncells'),
                            ds_chr.ddt_temp_radsw_fromflux - ds_chr.ddt_temp_radswclr_fromflux)

        ds_chr = ds_chr.drop_vars(('ddt_temp_radlw_fromflux',
                                   'ddt_temp_radlwclr_fromflux',
                                   'ddt_temp_radsw_fromflux',
                                   'ddt_temp_radswclr_fromflux'))
        
        ##########################################################################
        # read cloud classes
        print('      read cloud class')
        # read cloud class: 
        # cloud class stored every 30 minutes; cloud heating rates stored every hour
        # -> resample cloud class to get it every hour
        da_cclass = xr.open_dataset(ipath_cloudmask + 'nawdexnwp_' + resolution + \
                                    '_cloudclass_mis_' + sim + '.nc').rename({'clch': 'cclass'}).resample(time="1H").nearest(tolerance="5M")['cclass']
  
        # get days that are available for all resolutions
        da_cclass = select_analysis_days(da_cclass, expid)
        
        #print('cloud class shape and time')
        #print(da_cclass.values.shape)
        #print(da_cclass.time.values[0:6])
        #print(da_cclass.time.values[-6:])
        
        ##########################################################################
        # calculate time mean and area mean cloud heating rates for cloud classes
        print('      calculate area mean and time mean for cloud classes')

        # calculate time mean and area mean cloud cover for cloud classes
        print('      do means for cloud classes')
        for c, clas in enumerate(cclasses):
            #print('   ', clas)
            
            # V1: first area mean, then time mean
            # weighted area mean
            ds1 = ds_chr.where(da_cclass == c+1).weighted(weights).mean('ncells')
            # time mean
            ds_mean = ds1.mean('time')
            del ds1
            # END V1
            
            ## V2: first time mean, then area mean
            ## time mean
            #ds_mean = ds_chr.where(da_cclass == c+1).mean('time')
            ## weighted area mean
            #ds_mean = ds_mean.weighted(weights).mean('ncells')
            ## END V2
            
            # store data in dataset            
            if c == 0:
                ds_meanall = ds_mean.rename({'chr_lw': 'chr_lw_cclass' + str(c+1),
                                             'chr_sw': 'chr_sw_cclass' + str(c+1)})
            else:
                ds_meanall = ds_meanall.update(ds_mean.rename({'chr_lw': 'chr_lw_cclass' + str(c+1),
                                                               'chr_sw': 'chr_sw_cclass' + str(c+1)}))
            
            del ds_mean
        del c, clas

        # save means at nc file
        print('      save data')
        ofile = expid + \
                '_cloudheatingrate_cloudclasses_timemean_areamean_oceanmask_applied_amtm.nc'
        ds_meanall.to_netcdf(path=opath + ofile)#, format='NETCDF3_64BIT')#, unlimited_dims='time')
        
        del ds_meanall, ofile
        
        ##########################################################################
        del index, weights
        del expid
        del da_cclass, ds_chr
    del sim

del resolution
print('Done!')

80km
   0001
      read heating rates
      read cloud class
      calculate area mean and time mean for cloud classes
      save data
   0002
      read heating rates
      read cloud class
      calculate area mean and time mean for cloud classes
      save data
   0003
      read heating rates
      read cloud class
      calculate area mean and time mean for cloud classes
      save data
   0004
      read heating rates
      read cloud class
      calculate area mean and time mean for cloud classes
      save data
   0005
      read heating rates
      read cloud class
      calculate area mean and time mean for cloud classes
      save data
   0006
      read heating rates
      read cloud class
      calculate area mean and time mean for cloud classes
      save data
   0007
      read heating rates
      read cloud class
      calculate area mean and time mean for cloud classes
      save data
   0008
      read heating rates
      read cloud class
      calculate area mean and

In [5]:
'''###################################'''
'''      !!!!! VERSION 2 !!!!!        '''
'''store all cloud classes in one file'''
'''read all time steps at once        '''
'''    !!! WRONG AREA WEIGHTING !!!   '''
'''###################################'''

##############################################################################
opath = '/scratch/b/b380490/hackathon/'

ipath_grid = '/work/bb1018/icon_4_hackathon/'
ipath_oceanmask = '/work/bb1018/nawdex-hackathon_pp/'
ipath_cloudmask = '/pf/b/b380796/scratch/hackathon/george/'
ipath_cloudheat = '/work/bb1018/nawdex-hackathon_pp/ddttemp_rad-from-fluxes/'

#resolutions = ['80km', '40km', '20km', '10km', '5km']#, '2km']
resolutions = ['2km', '2km-shcon']
#gridres = ['R80000m', 'R40000m', 'R20000m', 'R10000m', 'R5000m']#, 'R2500m']
gridres = ['R2500m', 'R2500m']

# cloud classes
ncclass = 8 # number of cloud classes
cclasses = ['High (H)', 'Middle (M)', 'Low (L)',
            'H-M', 'M-L', 'H-L', 'H-M-L', 'clear sky']
cclass_save = ['H', 'M', 'L', 'H-M', 'M-L', 'H-L', 'H-M-L', 'clearsky']

for r, resolution in enumerate(resolutions):
    print(resolution)
    if resolution in ['80km', '40km', '20km']:
        sims = ['0001', '0002', '0003', '0004', '0005',
                '0006', '0007', '0008', '0009', '0010']
    elif resolution == '2km-shcon':
        sims = ['0001-shcon', '0002-shcon', '0005-shcon',
                '0006-shcon', '0007-shcon', '0008-shcon',
                '0009-shcon', '0010-shcon', '0011-shcon',
                '0012-shcon']
        resolution = '2km'
    else: # 10km, 5km, 2km
        sims = ['0001', '0002', '0003', '0004', '0005',
                '0006', '0007', '0008', '0009', '0010', '0011', '0012']
        
    for sim in sims:
        print('  ', sim)
        expid = 'nawdexnwp-' + resolution + '-mis-' + sim

        ##########################################################################
        # read ocean mask
        da_ocean = xr.open_dataset(ipath_oceanmask + '/openoceanmask/' + expid + \
                                   '_openoceanmask.nc')['mask_openocean']
        index = np.where(da_ocean == 1)[0]
        del da_ocean

        ##########################################################################
        # read cell area
        da_cell_area = xr.open_dataset(ipath_grid + '/grids/icon-grid_nawdex_78w40e23n80n_' + \
                                       gridres[r] + '.nc')['cell_area'].rename({'cell': 'ncells'})

        # weight for area mean
        weights = da_cell_area / (da_cell_area).sum(dim=['ncells'])

        # apply open ocean mask
        weights = weights.isel(ncells=index)

        del da_cell_area

        ##########################################################################
        # read heating rates
        print('      read heating rates')

        ds_chr = xr.open_zarr(ipath_cloudheat + expid + \
                              '_ddttemp_rad-from-fluxes_DOM01_ML.zarr')

        # remove first day from dataset
        #ds_chr = drop_first_day(ds_chr)
        
        # get days that are available for all resolutions
        ds_chr = select_analysis_days(ds_chr, expid)
        
        # apply open ocean mask
        ds_chr = ds_chr.isel(ncells=index)
        
        # get longwave and shortwave cloud-radiative heating rates by subtracting
        # all-sky and clear-sky values
        ds_chr['chr_lw'] = (('time', 'height', 'ncells'),
                            ds_chr.ddt_temp_radlw_fromflux - ds_chr.ddt_temp_radlwclr_fromflux)
        ds_chr['chr_sw'] = (('time', 'height', 'ncells'),
                            ds_chr.ddt_temp_radsw_fromflux - ds_chr.ddt_temp_radswclr_fromflux)

        ds_chr = ds_chr.drop_vars(('ddt_temp_radlw_fromflux',
                                   'ddt_temp_radlwclr_fromflux',
                                   'ddt_temp_radsw_fromflux',
                                   'ddt_temp_radswclr_fromflux'))
        
        ##########################################################################
        # read cloud classes
        print('      read cloud class')
        # read cloud class: 
        # cloud class stored every 30 minutes; cloud cover stored every hour
        # -> resample cloud class to get it every hour
        da_cclass = xr.open_dataset(ipath_cloudmask + 'nawdexnwp_' + resolution + \
                                    '_cloudclass_mis_' + sim + '.nc').rename({'clch': 'cclass'}).resample(time="1H").nearest(tolerance="5M")['cclass']
  
        # get days that are available for all resolutions
        da_cclass = select_analysis_days(da_cclass, expid)
        
        #print('cloud class shape and time')
        #print(da_cclass.values.shape)
        #print(da_cclass.time.values[0:6])
        #print(da_cclass.time.values[-6:])
        
        ##########################################################################
        # calculate time mean and area mean cloud cover for cloud classes
        print('      do means for cloud classes')
        for c, clas in enumerate(cclasses):
            #print('   ', clas)

            # time mean
            ds_mean = ds_chr.where(da_cclass == c+1).mean('time')
            
            # weighted area mean
            ds_mean = (ds_mean*weights).sum(dim='ncells')
            
            # store data in dataset            
            if c == 0:
                ds_meanall = ds_mean.rename({'chr_lw': 'chr_lw_cclass' + str(c+1),
                                             'chr_sw': 'chr_sw_cclass' + str(c+1)})
            else:
                ds_meanall = ds_meanall.update(ds_mean.rename({'chr_lw': 'chr_lw_cclass' + str(c+1),
                                                               'chr_sw': 'chr_sw_cclass' + str(c+1)}))
            
            del ds_mean
        del c, clas

        # save means at nc file
        print('      save data')
        ofile = expid + \
                '_cloudheatingrate_cloudclasses_timemean_areamean_oceanmask_applied.nc'
        #if resolution = '2km':
        #    ds_meanall.to_netcdf(path=opath + ofile, format='NETCDF3_64BIT', compute=False)#, unlimited_dims='time')
        #else:
        ds_meanall.to_netcdf(path=opath + ofile)#, format='NETCDF3_64BIT')#, unlimited_dims='time')
        
        del ds_meanall, ofile
    
        ##########################################################################
        del index, weights
        del expid
        del da_cclass, ds_chr
    del sim

del resolution
print('Done!')


2km
   0001
      read heating rates


MemoryError: Unable to allocate 66.2 GiB for an array with shape (72, 75, 3288831) and data type float32

In [4]:
'''############################################'''
'''           !!!!! VERSION 1 !!!!!            '''
'''store each cloud class in an individual file'''
'''read all time steps at once                 '''
'''        !!! WRONG AREA WEIGHTING !!!        '''
'''############################################'''

##############################################################################
opath = '/scratch/b/b380490/hackathon/'

ipath_grid = '/work/bb1018/icon_4_hackathon/'
ipath_oceanmask = '/work/bb1018/nawdex-hackathon_pp/'
ipath_cloudheat = '/work/bb1018/nawdex-hackathon_pp/ddttemp_rad-from-fluxes/'
ipath_cloudmask = '/pf/b/b380796/scratch/hackathon/george/'

resolutions = ['80km', '40km', '20km', '10km']#, '5km', '2km']
#resolutions = ['10km', '5km']#, '2km']
gridres = ['R80000m', 'R40000m', 'R20000m', 'R10000m']#, 'R5000m', 'R2500m']
#gridres = ['R10000m', 'R5000m']#, 'R2500m']

# cloud classes
ncclass = 8 # number of cloud classes
cclasses = ['High (H)', 'Middle (M)', 'Low (L)',
            'H-M', 'M-L', 'H-L', 'H-M-L', 'clear sky']
cclass_save = ['H', 'M', 'L', 'H-M', 'M-L', 'H-L', 'H-M-L', 'clearsky']

for r, resolution in enumerate(resolutions):
    print(resolution)
    if resolution in ['80km', '40km', '20km']:
        sims = ['0001', '0002', '0003', '0004', '0005',
                '0006', '0007', '0008', '0009', '0010']
    else:
        sims = ['0001', '0002', '0003', '0004', '0005',
                '0006', '0007', '0008', '0009', '0010', '0011', '0012']
    
    for sim in sims:
        print('  ', sim)
        expid = 'nawdexnwp-' + resolution + '-mis-' + sim

        ##########################################################################
        # read ocean mask
        da_ocean = xr.open_dataset(ipath_oceanmask + '/openoceanmask/' + expid + \
                                   '_openoceanmask.nc')['mask_openocean']
        index = np.where(da_ocean == 1)[0]
        del da_ocean

        ##########################################################################
        # read cell area
        da_cell_area = xr.open_dataset(ipath_grid + '/grids/icon-grid_nawdex_78w40e23n80n_' + \
                                       gridres[r] + '.nc')['cell_area'].rename({'cell': 'ncells'})

        # weight for area mean
        weights = da_cell_area / (da_cell_area).sum(dim=['ncells'])

        # apply open ocean mask
        weights = weights.isel(ncells=index)

        del da_cell_area

        ##########################################################################
        # read heating rates
        print('   read heating rates')

        ds_chr = xr.open_zarr(ipath_cloudheat + expid + \
                              '_ddttemp_rad-from-fluxes_DOM01_ML.zarr')

        # get longwave and shortwave cloud-radiative heating rates by subtracting
        # all-sky and clear-sky values
        ds_chr['chr_lw'] = (('time', 'height', 'ncells'),
                            ds_chr.ddt_temp_radlw_fromflux - ds_chr.ddt_temp_radlwclr_fromflux)
        ds_chr['chr_sw'] = (('time', 'height', 'ncells'),
                            ds_chr.ddt_temp_radsw_fromflux - ds_chr.ddt_temp_radswclr_fromflux)

        ds_chr = ds_chr.drop_vars(('ddt_temp_radlw_fromflux',
                                   'ddt_temp_radlwclr_fromflux',
                                   'ddt_temp_radsw_fromflux',
                                   'ddt_temp_radswclr_fromflux'))

        
        #print(ds_chr.chr_sw.shape)
        
        # remove first day from dataset
        ds_chr = drop_first_day(ds_chr)
        
        #print(ds_chr.time.values[0:6])
        
        # apply open ocean mask
        ds_chr = ds_chr.isel(ncells=index)

        #print('heating rate shape and time')
        #print(ds_chr.chr_lw.shape, ds_chr.chr_sw.shape)
        #print(ds_chr.time.values[0:6])
        #print(ds_chr.time.values[-6:])
        #print()
        
        ##########################################################################
        # read cloud classes
        print('   read cloud class')
        ds_cc = xr.open_dataset(ipath_cloudmask + 'nawdexnwp_' + resolution + \
                                '_cloudclass_mis_' + sim + '.nc').rename({'clch': 'cclass'})
                            # old data:
                                #'cloud_class_array_thres10p_' + \
                                #resolution + '_alltimesteps_v3.nc').rename({'clch': 'cclass'})

        # cloud class stored every 30 minutes. heating rates stored every hour
        # store cloud class also at every hour
        da_cclass = ds_cc.cclass[::2]
        del ds_cc

        #print('cloud class shape and time')
        #print(da_cclass.values.shape)
        #print(da_cclass.time.values[0:6])
        #print(da_cclass.time.values[-6:])

        ##########################################################################
        # calculate time mean and area mean cloud-radiative heating rates for
        # cloud classes
        for c, clas in enumerate(cclasses):
            print('   ', clas)

            # time mean
            ds_mean = ds_chr.where(da_cclass == c+1).mean('time')

            # weighted area mean
            ds_mean = (ds_mean*weights).sum(dim='ncells')

            # save means as nc file
            print('      save data')
            ofile = expid + '_' + cclass_save[c] + \
                    '_cloudheatingrate_timemean_areamean_oceanmask_applied.nc'
            ds_mean.to_netcdf(path=opath + ofile,
                            format='NETCDF3_64BIT', unlimited_dims='time')

            del ds_mean
        del c, clas
        ##########################################################################
        del index, weights
        del expid
        del da_cclass, ds_chr
        del ofile
    del sim

del resolution
print('Done!')

#del resolutions, ipath, opath

80km
   0001
   read cloud cover
(74, 75, 2799)
   read cloud class
(74, 2799)
    High (H)


/pf/b/b380459/conda-envs/Nawdex-Hackathon/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.50023783e-06
 8.19268622e-02 4.46608081e-01 1.00000369e+00 1.45094604e+00
 1.63207333e+00 1.71295320e+00 2.01719398e+00 2.73961354e+00
 3.79419472e+00 4.74484044e+00 5.22467088e+00 5.47363532e+00
 5.48961762e+00 5.51647800e+00 5.26350711e+00 4.77548468e+00
 4.29944390e+00 3.87551133e+00 3.49853084e+00 2.99425991e+00
 2.14566890e+00 9.08643078e-01 4.65628987e-02 4.81866388e-03
 3.14369699e-03 2.41596221e-03 2.91579410e-03 5.40430590e-03
 4.41133565e-03 5.34820722e-03 4.21870202e-03 4.33559072e-03
 3.39874191e-03 2.77488320e-03 3.40721366e-03 3.40629594e-03
 5.29503458e-03 1.10128715e-02 1.54111936e-02 1.47780874e-02
 1.32032885e-02 1.51932567e-02 1.34013785e-02 1.75375964e-02
 3.45255092e-02 4.43245110e-02 5.49496885e-02 7.92526027e-02
 8.77336293e-02 8.876403

/pf/b/b380459/conda-envs/Nawdex-Hackathon/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 2.66618398e-03 6.58885537e-03 1.48243639e-03
 8.26966861e-04 5.03594026e-04 6.02268040e-04 2.18669574e-03
 1.39805319e-03 6.82379714e-04 3.17123780e-03 3.12099565e-03
 5.69651386e-03 4.92440499e-03 5.66687687e-03 7.28047753e-03
 7.90544563e-03 6.43472770e-03 6.90807272e-03 8.42058664e-03
 1.07541353e-02 3.88973070e-02 5.66238020e-01 9.64120696e-01
 1.11221890e+00 1.26463071e+00 1.44500815e+00 1.59967510e+00
 1.51224909e+00 1.65843358e+00 1.32361790e+00 1.34512616e+00
 1.02942276e+00 1.07304784e+00 7.96795807e-01 7.20576761e-01
 5.60778167e-01 6.37104704e-01 6.08727304e-01 4.29658605e-01
 4.32880439e-01 3.32055594e-01 6.09521409e-02 4.94707097e-02
 4.17664009e-02 3.71319341e-02 3.62324848e-02 4.87089034e-02
 4.36705891e-02 4.130171

...

/pf/b/b380459/conda-envs/Nawdex-Hackathon/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 3.51275241e-04 1.06588737e-02
 1.11312655e-01 5.31406413e-01 1.10024543e+00 1.65136962e+00
 2.11739561e+00 2.46907843e+00 2.89064611e+00 3.24244857e+00
 4.13733577e+00 5.02357958e+00 5.67060134e+00 6.19083474e+00
 6.51166044e+00 6.87444340e+00 7.04826371e+00 7.35614024e+00
 7.34122985e+00 6.85837753e+00 6.26964430e+00 5.18310453e+00
 3.54823892e+00 1.42013353e+00 8.12231165e-02 2.09622982e-02
 1.60403484e-02 1.55498429e-02 1.58638874e-02 1.69526724e-02
 1.82628265e-02 2.02438748e-02 2.15849041e-02 2.23509584e-02
 2.04334721e-02 1.95386257e-02 1.87505315e-02 2.24562190e-02
 2.29024446e-02 3.02678803e-02 3.28793463e-02 3.71251135e-02
 4.91409873e-02 8.25795373e-02 5.20505926e-01 1.33477085e+00
 1.73734226e+00 2.03920369e+00 2.44224006e+00 2.71112290e+00
 2.68588388e+00 2.738610

/pf/b/b380459/conda-envs/Nawdex-Hackathon/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 5.29075501e-03 4.07148375e-02 2.08837294e-01
 4.75727492e-01 9.75194264e-01 1.59440500e+00 2.10119908e+00
 2.63737696e+00 3.18475743e+00 3.89870240e+00 4.94239951e+00
 6.42696479e+00 7.72582542e+00 8.71658682e+00 9.42751960e+00
 9.92275822e+00 1.05857290e+01 1.14916528e+01 1.23105894e+01
 1.31482354e+01 1.35125638e+01 1.36937392e+01 1.37723592e+01
 1.37893201e+01 1.36989791e+01 1.29110422e+01 1.16404698e+01
 1.07257690e+01 1.00393381e+01 9.60449754e+00 9.14973187e+00
 8.75397043e+00 8.35447548e+00 7.91609549e+00 7.74595047e+00
 7.38154411e+00 7.02386807e+00 6.42564576e+00 6.21329607e+00
 6.08242144e+00 6.07758482e+00 6.03956862e+00 6.43948615e+00
 6.72639560e+00 7.49098367e+00 7.25041423e+00 7.78799157e+00
 7.21495622e+00 7.24807193e+00 6.44363519e+00 6.31115584e+00
 5.56701206e+00 5.659013

/pf/b/b380459/conda-envs/Nawdex-Hackathon/lib/python3.8/site-packages/dask/array/numpy_compat.py:41: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)
